# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-ce63ef0736-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Olympic Games explorative search") 

Consider the following exploratory information need:

> investigate the Olympic Games and find the main BGPs related to these events. Find all the editions of the Summer and Winter Games, compare the sports and disciplines that belong to the games, records and countries.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`       | Human         | node      |
| `wd:Q1189`    | Usain Bolt    | node      |





Also consider

```
wd:Q1189 ?p ?obj .
```

is the BGP to retrieve all **properties of Usain Bolt**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for Olympic Games

2. Return all the editions of the Summer Olympic Games (do not consider future Olympic Games) with the country where they were played (the result set must be a list of elements with edition IRI and label, and country IRI and label).

3. Find the countries which held more than 2 Olympic Games (both Summer and Winter Games, do not consider future Olympic Games) (the result set must be a list of triples country IRI, label and #edition held).

4. Consider the 2008 Summer Olympic Games. Identify all the sports played in this edition (the result set must be a list of couples sport IRI and label).

5. Consider the 2008 Summer Olympic Games. For each sport return the number of different disciplines (the result set must be a list of triples sport IRI, label and #disciplines).

6. Consider the 2008 Summer Olympic Games. Find the top-10 people who won more gold medals (the result set must be a list of triples athlete IRI, label and #gold medal).

7. Find athletes who won at least one gold medal in 4 different editions of the Summer Olympic Games (the result set must be a list of triples athlete IRI, label and #edition).

8. Consider all the edition of the Summer Olympic Games. For each sport return the athlete who won more gold medal (the result set must be a list of elements with sport IRI and label, athlete IRI and label, and #gold medal). 

## Task 1
Identify the BGP for Olympic Games

In [17]:
# query example
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   #wd:Q1189 ?p ?obj . #we get Usain Bolt data
   #wd:Q8567 ?p ?obj. #we get 2008 "Summer Olympic" by "Usain Bolt"-"participant" relation.
   #wd:Q159821 ?p ?obj. #we get "Summer Olympic Games" by "Summer Olympic"-"instance of" relation.
   wd:Q159821 wdt:P279 ?obj.
   
   # get the label
   ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5389'), ('name', 'Olympic Games')]
1


Final query for this task

In [18]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q5389 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1546'), ('name', 'motto')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('name', 'flag')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1807

## Task 2
Return all the editions of the Summer Olympic Games (do not consider future Olympic Games) with the country where they were played (the result set must be a list of elements with edition IRI and label, and country IRI and label).

In [20]:
# write your queries
#we have the "Summer Olympic Games" data from previous code
queryString = """
SELECT DISTINCT ?countries ?name
WHERE {
   # bind something
   #wd:Q159821 ?p ?obj. #getting "Summer Olympic Games" data
   wd:Q159821 wdt:P527 ?obj. #getting all the editions of the Summer Olympic Games by "Summer Olympic Games"-"has parts" relation.
   ?obj wdt:P17 ?countries. #getting country data of the games for each edition of summer olympic games.  
   # get the label
   ?countries sc:name ?name.
}limit 20
"""

print("Results")
x=run_query(queryString)


Results
[('countries', 'http://www.wikidata.org/entity/Q1206012'), ('name', 'German Reich')]
[('countries', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('countries', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('countries', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China")]
[('countries', 'http://www.wikidata.org/entity/Q15180'), ('name', 'Soviet Union')]
[('countries', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil')]
[('countries', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada')]
[('countries', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('countries', 'http://www.wikidata.org/entity/Q174193'), ('name', 'United Kingdom of Great Britain and Ireland')]
[('countries', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('countries', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain')]
[('countries', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of Amer

Final query for this task

In [21]:
# write your final query# 
queryString = """
SELECT DISTINCT ?countries ?label ?nameLabel ?nameCountry
WHERE {
   wd:Q159821 wdt:P527 ?label. #getting all the editions of the Summer Olympic Games by "Summer Olympic Games"-"has parts" relation.
   ?label wdt:P17 ?countries. #getting country data of the games for each edition of summer olympic games.  
   # get the label
   ?label sc:name ?nameLabel.
   ?countries sc:name ?nameCountry.
}limit 20
"""

print("Results")
x=run_query(queryString)


Results
[('countries', 'http://www.wikidata.org/entity/Q1206012'), ('label', 'http://www.wikidata.org/entity/Q8123'), ('nameLabel', '1916 Summer Olympics'), ('nameCountry', 'German Reich')]
[('countries', 'http://www.wikidata.org/entity/Q1206012'), ('label', 'http://www.wikidata.org/entity/Q8150'), ('nameLabel', '1936 Summer Olympics'), ('nameCountry', 'German Reich')]
[('countries', 'http://www.wikidata.org/entity/Q142'), ('label', 'http://www.wikidata.org/entity/Q995653'), ('nameLabel', '2024 Summer Olympics'), ('nameCountry', 'France')]
[('countries', 'http://www.wikidata.org/entity/Q142'), ('label', 'http://www.wikidata.org/entity/Q8088'), ('nameLabel', '1900 Summer Olympics'), ('nameCountry', 'France')]
[('countries', 'http://www.wikidata.org/entity/Q142'), ('label', 'http://www.wikidata.org/entity/Q8132'), ('nameLabel', '1924 Summer Olympics'), ('nameCountry', 'France')]
[('countries', 'http://www.wikidata.org/entity/Q145'), ('label', 'http://www.wikidata.org/entity/Q8376'), ('na

## Task 3
Find the countries which held more than 2 Olympic Games (both Summer and Winter Games, do not consider future Olympic Games) (the result set must be a list of triples country IRI, label and #edition held).

In [96]:
# write your queries 
queryString = """
SELECT DISTINCT  ?country ?countryname  (count(?g) as ?numberOfGames)
WHERE {
  ?game wdt:P279 wd:Q5389. #getting all the games which are subclass of "Olympic Games"
  ?g wdt:P31 ?game. #getting the game edditions by "instance of"-"games" relation
  ?g wdt:P17 ?country. #getting the countries
  filter regex (str(?gname),  '(Summer|Winter)' ). #filtering the games to get only Winter and Summer games.
  
  ?g sc:name ?gname.
  ?country sc:name ?countryname.
  
}group by ?country ?countryname
having (count(?g) >= 2)
limit 20
"""

print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryname', 'Canada'), ('numberOfGames', '3')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryname', 'Italy'), ('numberOfGames', '3')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryname', 'Brazil'), ('numberOfGames', '2')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryname', "People's Republic of China"), ('numberOfGames', '2')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryname', 'France'), ('numberOfGames', '6')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('numberOfGames', '2')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('countryname', 'German Reich'), ('numberOfGames', '3')]
[('country', 'http://www.wikidata.org/entity/Q884'), ('countryname', 'South Korea'), ('numberOfGames', '2')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryname', 'Japan'), ('numberOfGames', '6')]
[('country', 'http://www.w

Final query for this task

In [97]:
# write your final query
queryString = """
SELECT DISTINCT  ?country ?countryname  (count(?g) as ?numberOfGames)
WHERE {
  ?game wdt:P279 wd:Q5389. #getting all the games which are subclass of "Olympic Games"
  ?g wdt:P31 ?game. #getting the game edditions by "instance of"-"games" relation
  ?g wdt:P17 ?country. #getting the countries
  filter regex (str(?gname),  '(Summer|Winter)' ). #filtering the games to get only Winter and Summer games.
  
  ?g sc:name ?gname.
  ?country sc:name ?countryname.
  
}group by ?country ?countryname
having (count(?g) >= 2)
limit 20
"""

print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryname', 'Canada'), ('numberOfGames', '3')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryname', 'Italy'), ('numberOfGames', '3')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryname', 'Brazil'), ('numberOfGames', '2')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryname', "People's Republic of China"), ('numberOfGames', '2')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryname', 'France'), ('numberOfGames', '6')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('numberOfGames', '2')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('countryname', 'German Reich'), ('numberOfGames', '3')]
[('country', 'http://www.wikidata.org/entity/Q884'), ('countryname', 'South Korea'), ('numberOfGames', '2')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryname', 'Japan'), ('numberOfGames', '6')]
[('country', 'http://www.w

## Task 4
Consider the 2008 Summer Olympic Games. Identify all the sports played in this edition (the result set must be a list of couples sport IRI and label).

In [98]:
# write your queries
#2008 Summer Olympic Games data was found in previous codes
queryString = """
SELECT DISTINCT  ?playedGames ?gameNames
WHERE {
  wd:Q8567 wdt:P527 ?playedGames. #finding the games that was played by "2008 Summer Olympic Games"-"has parts" relation.
  ?playedGames sc:name ?gameNames.
  ?gameNames ?p ?obj. #we see the game datas so we can collect the regular name of the sport
}
limit 20
"""

print("Results")
x=run_query(queryString)


Results
[('playedGames', 'http://www.wikidata.org/entity/Q221132'), ('gameNames', 'field hockey at the 2008 Summer Olympics')]
[('playedGames', 'http://www.wikidata.org/entity/Q387526'), ('gameNames', 'table tennis at the 2008 Summer Olympics')]
[('playedGames', 'http://www.wikidata.org/entity/Q220654'), ('gameNames', 'wrestling at the 2008 Summer Olympics')]
[('playedGames', 'http://www.wikidata.org/entity/Q204844'), ('gameNames', 'swimming at the 2008 Summer Olympics')]
[('playedGames', 'http://www.wikidata.org/entity/Q220942'), ('gameNames', 'water polo at the 2008 Summer Olympics')]
[('playedGames', 'http://www.wikidata.org/entity/Q212887'), ('gameNames', 'weightlifting at the 2008 Summer Olympics')]
[('playedGames', 'http://www.wikidata.org/entity/Q220930'), ('gameNames', 'boxing at the 2008 Summer Olympics')]
[('playedGames', 'http://www.wikidata.org/entity/Q189816'), ('gameNames', 'basketball at the 2008 Summer Olympics')]
[('playedGames', 'http://www.wikidata.org/entity/Q202246

Final query for this task

In [99]:
# write your final query
#2008 Summer Olympic Games data was found in previous codes
queryString = """
SELECT DISTINCT  ?name ?gameNames
WHERE {
  wd:Q8567 wdt:P527 ?playedGames. #finding the games that was played by "2008 Summer Olympic Games"-"has parts" relation.
  ?playedGames wdt:P641 ?name.
  ?name sc:name ?gameNames.
  
}
limit 20
"""

print("Results")
x=run_query(queryString)


Results
[('name', 'http://www.wikidata.org/entity/Q10980'), ('gameNames', 'triathlon')]
[('name', 'http://www.wikidata.org/entity/Q213934'), ('gameNames', 'canoeing and kayaking')]
[('name', 'http://www.wikidata.org/entity/Q326827'), ('gameNames', 'artistic gymnastics')]
[('name', 'http://www.wikidata.org/entity/Q180692'), ('gameNames', 'synchronized swimming')]
[('name', 'http://www.wikidata.org/entity/Q465014'), ('gameNames', 'trampolining')]
[('name', 'http://www.wikidata.org/entity/Q1455'), ('gameNames', 'field hockey')]
[('name', 'http://www.wikidata.org/entity/Q1734'), ('gameNames', 'volleyball')]
[('name', 'http://www.wikidata.org/entity/Q206989'), ('gameNames', 'shooting sport')]
[('name', 'http://www.wikidata.org/entity/Q2736'), ('gameNames', 'association football')]
[('name', 'http://www.wikidata.org/entity/Q31920'), ('gameNames', 'swimming')]
[('name', 'http://www.wikidata.org/entity/Q32112'), ('gameNames', 'boxing')]
[('name', 'http://www.wikidata.org/entity/Q4543'), ('game

## Task 5
Consider the 2008 Summer Olympic Games. For each sport return the number of different disciplines (the result set must be a list of triples sport IRI, label and #disciplines).

In [109]:
#2008 Summer Olympic Games data was found in previous codes
queryString = """
SELECT DISTINCT  (count(?disc) as ?disciplineNum) ?name ?sportname
WHERE {
  wd:Q8567 wdt:P527 ?playedGames. #finding the games that was played by "2008 Summer Olympic Games"-"has parts" relation.
  ?playedGames wdt:P641 ?name.
  ?name wdt:P527 ?disc.  #after finding the names of the games we find the related disciplines by using sports name-"has parts" relation
  ?name sc:name ?sportname.
  
}group by ?name ?sportname
limit 20
"""

print("Results")
x=run_query(queryString)


Results
[('disciplineNum', '4'), ('name', 'http://www.wikidata.org/entity/Q1734'), ('sportname', 'volleyball')]
[('disciplineNum', '6'), ('name', 'http://www.wikidata.org/entity/Q43450'), ('sportname', 'gymnastics')]
[('disciplineNum', '2'), ('name', 'http://www.wikidata.org/entity/Q32112'), ('sportname', 'boxing')]
[('disciplineNum', '1'), ('name', 'http://www.wikidata.org/entity/Q5369'), ('sportname', 'baseball')]
[('disciplineNum', '2'), ('name', 'http://www.wikidata.org/entity/Q213934'), ('sportname', 'canoeing and kayaking')]
[('disciplineNum', '8'), ('name', 'http://www.wikidata.org/entity/Q542'), ('sportname', 'athletics')]
[('disciplineNum', '3'), ('name', 'http://www.wikidata.org/entity/Q10980'), ('sportname', 'triathlon')]
[('disciplineNum', '5'), ('name', 'http://www.wikidata.org/entity/Q32485'), ('sportname', 'modern pentathlon')]
8


Final query for this task

In [ ]:
# write your final query
queryString = """
SELECT DISTINCT  (count(?disc) as ?disciplineNum) ?name ?sportname
WHERE {
  wd:Q8567 wdt:P527 ?playedGames. #finding the games that was played by "2008 Summer Olympic Games"-"has parts" relation.
  ?playedGames wdt:P641 ?name.
  ?name wdt:P527 ?disc.  #after finding the names of the games we find the related disciplines by using sports name-"has parts" relation
  ?name sc:name ?sportname.
  
}group by ?name ?sportname
limit 20
"""

print("Results")
x=run_query(queryString)


## Task 6
Consider the 2008 Summer Olympic Games. Find the top-10 people who won more gold medals (the result set must be a list of triples athlete IRI, label and #gold medal).

In [204]:
queryString = """ #first we find all the people who participated in "2008 Summer Olympic Games"
SELECT DISTINCT   ?o ?objname
WHERE {

    #wd:Q8567 wdt:P710 ?participants . #all the participants who was in "2008 Summer Olympic Games"
    ?participants wdt:P1344 wd:Q8567. 
    #?participants wdt:P1346 wd:Q15243387.
    
    wd:1346 wdt:1855 ?o.
    #?s ?p wd:Q15243387.

    #?editions wdt:P166 ?obj. #we see all the awards by "participated in"-"awarded" relation.
    #?athlete wdt:P1344 ?editions . 
    #?edition wdt:P1344 wdt:P166 wd:Q15243387.
    
    #?s sc:name ?athletename .
    ?o sc:name ?objname.
} limit 20

"""

print("Results")
x=run_query(queryString)


Results
Empty


Final query for this task

In [140]:
# write your final query

# write your queries


queryString = """
SELECT DISTINCT   ?s ?name
WHERE {
   ?s wdt:P2522 wd:Q8567.
  ?s sc:name ?name.
  
}limit 300
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q126119'), ('name', 'Michał Jeliński')]
[('s', 'http://www.wikidata.org/entity/Q10058'), ('name', 'Justine Smethurst')]
[('s', 'http://www.wikidata.org/entity/Q119562'), ('name', 'Sergio Agüero')]
[('s', 'http://www.wikidata.org/entity/Q1150190'), ('name', 'Cédric Burdet')]
[('s', 'http://www.wikidata.org/entity/Q1094035'), ('name', 'Gemma Beadsworth')]
[('s', 'http://www.wikidata.org/entity/Q1283673'), ('name', 'Steve Williams')]
[('s', 'http://www.wikidata.org/entity/Q1042890'), ('name', 'Carlos Espínola')]
[('s', 'http://www.wikidata.org/entity/Q1281809'), ('name', 'Slobodan Soro')]
[('s', 'http://www.wikidata.org/entity/Q1267138'), ('name', 'Ruslan Tumenbaev')]
[('s', 'http://www.wikidata.org/entity/Q1174097'), ('name', 'David Crawshay')]
[('s', 'http://www.wikidata.org/entity/Q1097044'), ('name', 'Claude Onesta')]
[('s', 'http://www.wikidata.org/entity/Q127132'), ('name', 'Paddy Barnes')]
[('s', 'http://www.wikidata.org/entity/Q123275

## Task 7
Find athletes who won at least one gold medal in 4 different editions of the Summer Olympic Games (the result set must be a list of triples athlete IRI, label and #edition).

In [199]:
queryString = """
SELECT DISTINCT  ?athlete ?athletename
WHERE {

    ?solympıcs wdt:P279 wd:Q5389.
    ?solympıcs sc:name ?name .
    FILTER REGEX(STR(?name), 'Summer')
    ?edıtıons wdt:P31 ?solympıcs .
    ?athlete wdt:P1344 ?edıtıons .
    ?athlete sc:name ?athletename .
    

}limit 10
"""

print("Results")
x=run_query(queryString)




queryString = """ #first we find all the people who participated in "2008 Summer Olympic Games"
SELECT DISTINCT   ?editions ?athletename 
WHERE {

    ?solympics wdt:P279 wd:Q5389.
    ?solympics sc:name ?name .
    FILTER REGEX(STR(?name), 'Summer')
    ?editions wdt:P31 ?solympics .

    #?editions wdt:P166 ?obj. #we see all the awards by "participated in"-"awarded" relation.
    #?athlete wdt:P1344 ?editions . 
    #?edition wdt:P166 wd:Q15243387.
    
    ?editions sc:name ?athletename .
}
limit 20
"""

print("Results")
x=run_query(queryString)


Results
[('athlete', 'http://www.wikidata.org/entity/Q13639919'), ('athletename', 'Joseph Goubert')]
[('athlete', 'http://www.wikidata.org/entity/Q313195'), ('athletename', 'Archie Hahn')]
[('athlete', 'http://www.wikidata.org/entity/Q15870814'), ('athletename', 'Josef Klíma')]
[('athlete', 'http://www.wikidata.org/entity/Q16011595'), ('athletename', 'Jarl Johnsen')]
[('athlete', 'http://www.wikidata.org/entity/Q16013566'), ('athletename', 'Nancy Archibald')]
[('athlete', 'http://www.wikidata.org/entity/Q16014208'), ('athletename', 'Jan Vrolijk')]
[('athlete', 'http://www.wikidata.org/entity/Q16065294'), ('athletename', 'Vincens Grandjean')]
[('athlete', 'http://www.wikidata.org/entity/Q20852710'), ('athletename', 'Hans Spiegel')]
[('athlete', 'http://www.wikidata.org/entity/Q4063472'), ('athletename', 'Teodoro Alcalde')]
[('athlete', 'http://www.wikidata.org/entity/Q16204467'), ('athletename', 'Andrew Drew')]
10
Results
[('editions', 'http://www.wikidata.org/entity/Q8098'), ('athleten

Final query for this task

In [ ]:
# write your final query

## Task 8
Consider all the edition of the Summer Olympic Games. For each sport return the athlete who won more gold medal (the result set must be a list of elements with sport IRI and label, athlete IRI and label, and #gold medal).

In [ ]:
# write your queries

Final query for this task

In [197]:
# write your final query

# write your final query

# write your queries


queryString = """
SELECT DISTINCT   ?s ?p ?name ?pname
WHERE {
   ?s ?p wd:Q406039.
   ?s sc:name ?name.
   ?p sc:name ?pname.
  
}limit 10
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q16034582'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Parfums Millot'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q125409'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Charles Jacque'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q2512767'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Wessel Couzijn'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q19819372'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Claude Trèves'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q28464981'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Shelley Gautier'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q311374'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Alexander Ovechkin'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/e